In [5]:
import time
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
def string_to_timestamp(strings,t=48):
    """
    :param strings: [b'2013070101,b'2013070102']
    :param t: 48 timestamp of a day 
    :return: '2013-0701 00:00:00','2013-07-01 00:30:00'
    """
    timestamps = []
    time_per_slot = 24.0 / t
    num_per_t = t
    for t in strings:
        year, month, day, slot = int(t[:4]), int(t[4:6]), int(t[6:8]), int(t[8:]) - 1 
        time    
    